# 드라이브 연동

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
%cd /content/gdrive/My\ Drive

In [ ]:
#zip file을 내 드라이브에 다운받아야함
!mkdir ./beep_data
!unzip XY_train.zip -d ./beep_data
!unzip XY_test.zip -d ./beep_data

# Data Check

In [1]:
n_freq = 128  # 스펙트로그램 높이

# Data Loader
- with Keras

In [2]:
import os 
import pandas as pd
import numpy as np
from keras.utils import Sequence

In [4]:
data_dir = os.path.join('.','data', 'XY_train')
x_s = []
y_s = []
for file in os.listdir(data_dir):
    if file.startswith('x_'):
        x_s.append(os.path.join(data_dir,file))
    elif file.startswith('y_'):
        y_s.append(os.path.join(data_dir,file))
    x_s = sorted(x_s)
    y_s = sorted(y_s)
df = pd.DataFrame({'x':x_s, 'y':y_s})
df.head()

,x,y
0,.\data\XY_train\x_0.npy,.\data\XY_train\y_0.npy
1,.\data\XY_train\x_1.npy,.\data\XY_train\y_1.npy
2,.\data\XY_train\x_2.npy,.\data\XY_train\y_2.npy
3,.\data\XY_train\x_3.npy,.\data\XY_train\y_3.npy
4,.\data\XY_train\x_4.npy,.\data\XY_train\y_4.npy


In [5]:
train_ratio = 0.8
idxs = list(range(len(df)))
np.random.shuffle(idxs)
train_idx = idxs[:int(len(df)*train_ratio)]
valid_idx = idxs[int(len(df)*train_ratio):]

train_df = df.loc[train_idx]
valid_df = df.loc[valid_idx]

print(train_df.shape, valid_df.shape)

(8, 2) (2, 2)


In [7]:
data_dir = os.path.join('.','data', 'XY_test')
x_s = []
y_s = []
for file in os.listdir(data_dir):
    if file.startswith('x_'):
        x_s.append(os.path.join(data_dir,file))
    elif file.startswith('y_'):
        y_s.append(os.path.join(data_dir,file))
    x_s = sorted(x_s)
    y_s = sorted(y_s)
test_df = pd.DataFrame({'x':x_s, 'y':y_s})
test_df.head()

FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: '.\\data\\XY_test'

In [8]:
class DataGenerator(Sequence):
    def __init__(self, df, batch_size, shuffle = True):
        self.X = list(df.x)
        self.y = list(df.y)
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.X))
        if self.shuffle:
            np.random.shuffle(self.indexes)
            
    def __len__(self):
        return int(np.floor(len(self.X) / self.batch_size))
    
    def __data_generation(self, X_list, y_list):
        X = []
        y = []
        for i, (img, label) in enumerate(zip(X_list, y_list)):
            X.append(np.load(img))
            y.append(np.load(label))
        
        X = np.stack(X, axis=0)
        y = np.stack(y, axis=0)

        return X, y
        
    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size : (index + 1) * self.batch_size]
        X_list = [self.X[k] for k in indexes]
        y_list = [self.y[k] for k in indexes]
        X, y = self.__data_generation(X_list, y_list)
        return X, y

In [10]:
train_generator = DataGenerator(train_df, 5)
valid_generator = DataGenerator(valid_df, 5)
# test_generator = DataGenerator(test_df, 3)

# Train

In [11]:
from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from keras.optimizers import Adam

In [12]:
def make_model(input_shape):
    
    X_input = Input(shape = input_shape)
    X = Conv1D(196, kernel_size=15, strides=4)(X_input)         # CONV1D
    X = BatchNormalization()(X)                                 # Batch normalization
    X = Activation('relu')(X)                                   # ReLu activation
    X = Dropout(0.8)(X)                                         # dropout (use 0.8)

    X = GRU(units = 128, return_sequences = True)(X)            # GRU (use 128 units and return the sequences)
    X = Dropout(0.8)(X)                                         # dropout (use 0.8)
    X = BatchNormalization()(X)                                 # Batch normalization

    X = GRU(units = 128, return_sequences = True)(X)            # GRU (use 128 units and return the sequences)
    X = Dropout(0.8)(X)                                         # dropout (use 0.8)
    X = BatchNormalization()(X)                                 # Batch normalization
    X = Dropout(0.8)(X)                                         # dropout (use 0.8)

    X = TimeDistributed(Dense(1, activation = "sigmoid"))(X)    # time distributed  (sigmoid)

    model = Model(inputs = X_input, outputs = X)
    return model

In [13]:
model = make_model(input_shape = (None, n_freq))

In [14]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, 128)]       0         
                                                                 
 conv1d (Conv1D)             (None, None, 196)         376516    
                                                                 
 batch_normalization (BatchN  (None, None, 196)        784       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, None, 196)         0         
                                                                 
 dropout (Dropout)           (None, None, 196)         0         
                                                                 
 gru (GRU)                   (None, None, 128)         125184    
                                                             

In [15]:
opt = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])

c:\Users\user\anaconda3\envs\Beeeeep_2\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [16]:
model.fit_generator(generator=train_generator,
                    validation_data=valid_generator,
                    epochs = 10)

c:\Users\user\anaconda3\envs\Beeeeep_2\lib\site-packages\ipykernel_launcher.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  This is separate from the ipykernel package so we can avoid doing imports until


Epoch 1/10
1/1 [==============================] - 6s 6s/step - loss: 1.3901 - accuracy: 0.5104
Epoch 2/10
1/1 [==============================] - 0s 450ms/step - loss: 1.4265 - accuracy: 0.5113
Epoch 3/10
1/1 [==============================] - 0s 401ms/step - loss: 1.4681 - accuracy: 0.4906
Epoch 4/10
1/1 [==============================] - 1s 549ms/step - loss: 1.5193 - accuracy: 0.4802
Epoch 5/10
1/1 [==============================] - 0s 392ms/step - loss: 1.3829 - accuracy: 0.5066
Epoch 6/10
1/1 [==============================] - 0s 360ms/step - loss: 1.4335 - accuracy: 0.5019
Epoch 7/10
1/1 [==============================] - 0s 390ms/step - loss: 1.3832 - accuracy: 0.5009
Epoch 8/10
1/1 [==============================] - 0s 342ms/step - loss: 1.3469 - accuracy: 0.5217
Epoch 9/10
1/1 [==============================] - 0s 350ms/step - loss: 1.4215 - accuracy: 0.4943
Epoch 10/10
1/1 [==============================] - 0s 318ms/step - loss: 1.4709 - accuracy: 0.4651


# Test

In [ ]:
def postprocessing(outputs, th):
    for output in outputs:
        output[output<th] = 0
        output[output>=th] = 1
    return outputs

In [ ]:
import matplotlib.pyplot as plt
for batch in test_generator:
    x, y = batch
    output = model.predict(x)
    output = postprocessing(output,0.6)
    for i in range(len(y)):
        plt.plot(y[i], label='true')
        plt.plot(output[i], label='predict')
        plt.legend()
        plt.show()